# Instruction

Create a private repo on github called calscrape. Share it with mdogy. Create an ipython notebook and commit it. I want you to keep committing it as you edit it so I can see it build up. Your mission is to scrape the cuny fall 2018 academic calendar site using the requests library, beautiful soup and pandas, just like in class. You should wind up with a pandas data frame where the index column is a python date. There should be a column for "day of the week" with variable lable dow and a column called text with the explanation. <https://www.ccny.cuny.edu/registrar/fall-2018-academic-calendar>

Print out this data frame. 


**5 extra credit points if you figure out how to use the google api to programatically add these dates to your google calendar with the request library.**


## Plan of Execution
1. use request to get the data using the link
2. pass the request txt to beautifulsoup to prettify or parse the html file
3. use the find_all function in BeautifulSoup to fetch for the rows and cols of interes

In [1]:
import pandas as pd
import requests
from IPython.display import HTML
from bs4 import BeautifulSoup as BS

## Step 1: Use Request to get the web text

In [2]:
req = requests.get("https://www.ccny.cuny.edu/registrar/fall-2018-academic-calendar")

## Step 2a: Pass to BeautifulSoup

In [3]:
soup = BS(req.text, 'html.parser') 

## Step 2b: Exploration of the data structure in Soup

In [4]:
#Find all the table
table = soup.find_all('table')

#find the number of table, there is just 1 table, td = each column, tr = each row
print(len(table))

1


In [30]:
rows = [row for row in table[0].find_all('tr')]
rows[0]

<tr><td style="width:134px;"><p><strong>DATES</strong></p></td><td style="width:124px;"><p><strong>DAYS</strong></p></td><td style="width:390px;"><p> </p></td></tr>

In [6]:
#column headers
header = [item.get_text().strip() for item in rows[0]] #'\xa0' for the third column if i dont do .strip()
header[1] = 'DAYS_OF_THE_WEEK'
header[2] = 'TEXT_WITH_EXPLANATIONS'
header

# in this case, you can just using one line to manually set for header
# header  = ['DATES', 'DAYS_OF_THE_WEEK', 'TEXT_WITH_EXPLANATIONS']

['DATES', 'DAYS_OF_THE_WEEK', 'TEXT_WITH_EXPLANATIONS']

In [45]:
#get text from each column of the row, starting from row 1

#1. find each col in rows[1:],starting from rows 1 onward --> [col for col in rows[1:]]
#2. In each col, get the text from the cell 
#      --> get text from each col by finding 'td'
#      --> item.get_text() for item in col.find_all('td')


data = [[item.get_text().strip() for item in col.find_all('td')] for col in rows[1:]]
# data

# using the lines will remove the header column, so you can directly generate df_cleaned in two cells below
# data = [[item.get_text().strip() for item in col.find_all('p')] for col in rows[1:]]
# data =  [item for item in data if item]

In [41]:
# data array is not what I want, I want the first item of each array to be its own list, and second item of the array in a second list

# the list(zip()) function does what I want, * means perform on itself
data_t = list(zip(*data)) 
print(data_t)
# I want to append the header to data_t
full_data = list(zip(header,data_t))
# full_data

[]


## Step 3: Construct a DataFrame from the Scrapped data using Pandas

In [11]:
# make a dictionary out of the full_data <class 'list'>
from datetime import datetime as dt
df = pd.DataFrame(dict(full_data))

df_remove_month = df[~df.DATES.str.contains("^August$|^September$|^October$|^November$|^December$|^January$")]
df_cleaned = pd.DataFrame(df_remove_month)
# df_cleaned

,DATES,DAYS_OF_THE_WEEK,TEXT_WITH_EXPLANATIONS
1,August 01,Wednesday,Application for degree for January and Februar...
2,August 20,Monday,Last day to apply for Study Abroad
3,August 26,Sunday,Last day of Registration;Last day to file ePer...
4,August 27,Monday,Classes begin;Initial Registration Appeals beg...
5,August 27 –September 02,Monday –Sunday,Change of program period; late fees apply
6,August 28,Tuesday,Last day to submit a request for Independent S...
8,September 01,Saturday,First day of Saturday Classes
9,September 02,Sunday,Last day to add a class to an existing enrollm...
10,September 03,Monday,Course Withdrawal drop period begins (A grade ...
11,September 05,Wednesday,Classes follow a Monday schedule


In [46]:
t = df_cleaned.DATES.str.replace("-", " –").str.split(" –")
start = []
end = []

#to parse out the date into start and end date
for x in t[:-1]:
    if len(x) > 1:
        start.append(x[0] + " 2018") 
        end.append(x[1] + " 2018")
    else:
        start.append(x[0] +  " 2018")
        end.append(x[0] + " 2018")

#fix some small inconsistency in writing dates
start.append("January 1 2019")
end.append("January 1 2019")
end[-4] = 'December 25 2018'

In [48]:
#add to the current dataframe
df_cleaned['START_DATE'] = start
df_cleaned['END_DATE'] = end

#convert to time format
df_cleaned['START_DATE'] = pd.to_datetime(df_cleaned['START_DATE'], format = '%B %d %Y')
df_cleaned['END_DATE'] = pd.to_datetime(df_cleaned['END_DATE'], format = '%B %d %Y')

# using below code can replace the above code, don't need to set twice
# df_cleaned['START_DATE'] = pd.to_datetime(start, format = '%B %d %Y')
# df_cleaned['END_DATE'] = pd.to_datetime(end, format = '%B %d %Y')

#create a new dataframe with re-ordered columns
DF = pd.DataFrame(df_cleaned[["DATES","START_DATE","END_DATE","DAYS_OF_THE_WEEK","TEXT_WITH_EXPLANATIONS"]])

In [49]:
#convert to datetime format
DF.set_index('DATES')


,START_DATE,END_DATE,DAYS_OF_THE_WEEK,TEXT_WITH_EXPLANATIONS
DATES,,,,
August 01,2018-08-01,2018-08-01,Wednesday,Application for degree for January and Februar...
August 20,2018-08-20,2018-08-20,Monday,Last day to apply for Study Abroad
August 26,2018-08-26,2018-08-26,Sunday,Last day of Registration;Last day to file ePer...
August 27,2018-08-27,2018-08-27,Monday,Classes begin;Initial Registration Appeals beg...
August 27 –September 02,2018-08-27,2018-09-02,Monday –Sunday,Change of program period; late fees apply
August 28,2018-08-28,2018-08-28,Tuesday,Last day to submit a request for Independent S...
September 01,2018-09-01,2018-09-01,Saturday,First day of Saturday Classes
September 02,2018-09-02,2018-09-02,Sunday,Last day to add a class to an existing enrollm...
September 03,2018-09-03,2018-09-03,Monday,Course Withdrawal drop period begins (A grade ...


## Extra Credit
- use this Google Calender API: https://developers.google.com/calendar/quickstart/python
- 5 extra credit points if you figure out how to use the google api to programatically add these dates to your google calendar with the request library.

In [257]:
from __future__ import print_function
import datetime
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/calendar']


def main():
    """Shows basic usage of the Google Calendar API.
    Prints the start and name of the next 10 events on the user's calendar.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('calendar', 'v3', credentials=creds)

    
# end time should not be same time as start time, 
# now the hour in the date is set to 00:00 am for both start and end date
# you can edit end date hour below
    # endtime = rows.END_DATE + datetime.timedelta(hours=23, minutes=59) 
    # use in the below call for end: 'dateTime': endtime.isoformat(),
                
    # Call the Calendar API by iterrating over the rows
    for index, rows in DF.iterrows():
        event = {
            'summary' : rows.TEXT_WITH_EXPLANATIONS,
            'start' : {
                'dateTime': rows.START_DATE.isoformat(),
                'timeZone': 'America/New_York',
            },
            'end' : {
                'dateTime': rows.END_DATE.isoformat(),
                'timeZone': 'America/New_York',
            },
  
                }
        event = service.events().insert(calendarId='primary', body=event).execute()
        print('Event Created: %s' % event['summary'])
            
if __name__ == '__main__':
    main()

Event Created: Application for degree for January and February 2019 begins;
Event Created: Last day to apply for Study Abroad
Event Created: Last day of Registration;Last day to file ePermit request for Fall 2018;
Event Created: Classes begin;Initial Registration Appeals begin;Last day to drop classes for 100% tuition refund;
Event Created: Change of program period; late fees apply
Event Created: Last day to submit a request for Independent Study
Event Created: First day of Saturday Classes
Event Created: Last day to add a class to an existing enrollment;Last day for 75% tuition refund;Financial Aid Certification Enrollment Status date;Last day to apply for Audit option;Last day to drop without the grade of WD;Initial Registration Appeals end;
Event Created: Course Withdrawal drop period begins (A grade of “WD” is assigned to students who officially drop a class);Verification of Enrollment Rosters available to faculty;College Closed;
Event Created: Classes follow a Monday schedule
Even